In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import torch

In [ ]:
# Function to parse YOLO annotation file and extract bounding box coordinates
def parse_annotation_file(annotation_file_path):
    with open(annotation_file_path, 'r') as file:
        lines = file.readlines()

    # Extract bounding box coordinates and convert to numpy array
    # Format: [class_id, x_min, y_min, x_max, y_max]
    bounding_boxes = [list(map(float, line.strip().split())) for line in lines]

    return np.array(bounding_boxes)

In [ ]:
# Replace 'your_data_folder' with the path to the folder containing YOLO annotation files
data_folder = '/content/drive/MyDrive/Data2/validation_data_234Images/M_R_H_NH'

In [63]:
# Lists to store input and output bounding boxes
input_boxes = []
output_boxes = []
image_width=0
# Loop through all YOLO annotation files in the data folder
for file_name in os.listdir(data_folder):
    if file_name.lower().endswith('.jpg'):
        file_path = os.path.join(data_folder, file_name)
        image = cv2.imread(file_path)
        height, width = image.shape[:2]
        image_width=width
    if file_name.lower().endswith('.txt'):
        annotation_file_path = os.path.join(data_folder, file_name)
        helmet_bbox = parse_annotation_file(annotation_file_path)
        helmet_bbox=helmet_bbox.tolist()
        # print(helmet_bbox)
        input_box=[]
        output_box=[]
        # Assuming the helmet/no-helmet class is 0 (modify this if different)
        for rows in helmet_bbox:
          # print(type(rows))
          if rows[0]==3 or rows[0]==2:
              # Extract the corresponding rider bounding box from the helmet/no-helmet bounding box
              rider_bbox = rows.copy()
              rider_bbox[0] = 1  # Assuming the rider class is 1 (modify this if different)

              # Append the bounding boxes to the input and output lists
              input_box.append(rows)  # Extract coordinates excluding the class ID
              output_box.append(rider_bbox)  # Extract coordinates excluding the class ID
        # input_box = tf.ragged.constant(input_box)
        # output_box = tf.ragged.constant(output_box)
        input_boxes.append(input_box)
        output_boxes.append(output_box)
input_boxes = tf.ragged.constant(input_boxes)
output_boxes = tf.ragged.constant(output_boxes)
# input_boxes=tf.convert_to_tensor(input_boxes)
# output_boxes=tf.convert_to_tensor(output_boxes)

In [64]:
print(input_boxes)

<tf.RaggedTensor [[[2.0, 0.2875, 0.532407, 0.125, 0.216667],
  [2.0, 0.840104, 0.323611, 0.013542, 0.017593],
  [2.0, 0.632292, 0.314815, 0.010417, 0.014815]],
 [[2.0, 0.467187, 0.49213, 0.077083, 0.130556],
  [2.0, 0.123438, 0.398611, 0.030208, 0.052778]],
 [[2.0, 0.7125, 0.564815, 0.095833, 0.157407],
  [2.0, 0.234896, 0.442593, 0.029167, 0.059259]],
 [[2.0, 0.532813, 0.496296, 0.052083, 0.094444]],
 [[3.0, 0.446875, 0.472685, 0.120833, 0.232407]],
 [[2.0, 0.232552, 0.522685, 0.085938, 0.152778],
  [3.0, 0.067708, 0.426852, 0.008333, 0.012963]],
 [[2.0, 0.779948, 0.562963, 0.090104, 0.155556],
  [2.0, 0.933333, 0.752778, 0.133333, 0.412963]],
 [[2.0, 0.552865, 0.502778, 0.085938, 0.146296],
  [2.0, 0.411458, 0.460648, 0.044792, 0.086111]],
 [[2.0, 0.404167, 0.459259, 0.047917, 0.083333],
  [2.0, 0.54349, 0.49537, 0.069271, 0.146296],
  [3.0, 0.798698, 0.511574, 0.042188, 0.093519],
  [2.0, 0.929688, 0.54213, 0.067708, 0.130556]] ,
 [[3.0, 0.760677, 0.39213, 0.070312, 0.147222]],
 [[3

In [65]:
# Convert the lists of bounding boxes to numpy arrays
# input_boxes = np.array(input_boxes,dtype='object')
# output_boxes = np.array(output_boxes,dtype='object')

# input_boxes=input_boxes.reshape(input_boxes.shape[0],-1)
# output_boxes=output_boxes.reshape(output_boxes.shape[0],-1)

# input_boxes = list(map(torch.as_tensor, input_boxes))
# input_boxes=torch.nested.as_nested_tensor(input_boxes, dtype=torch.long)

# output_boxes = list(map(torch.as_tensor, output_boxes))
# output_boxes=torch.nested.as_nested_tensor(output_boxes, dtype=torch.long)
# Normalize the bounding box coordinates to the range [0, 1]
# input_boxes /= image_width  # Normalize by the image width
# output_boxes /= image_width

# print(type(input_boxes))

# input_boxes = tf.ragged.constant(input_boxes)
# output_boxes = tf.ragged.constant(output_boxes)
# input_boxes=tf.convert_to_tensor(input_boxes)
# output_boxes=tf.convert_to_tensor(output_boxes)

# input_boxes = torch.Tensor(input_boxes)
# output_boxes = torch.Tensor(output_boxes)

# Create the amodal regressor model
model = Sequential()
model.add(Dense(16, input_dim=4, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4))  # Output layer with 4 nodes for the bounding box coordinates

In [61]:
  print(input_boxes)

<tf.RaggedTensor [[[2.0, 0.2875, 0.532407, 0.125, 0.216667],
  [2.0, 0.840104, 0.323611, 0.013542, 0.017593],
  [2.0, 0.632292, 0.314815, 0.010417, 0.014815]],
 [[2.0, 0.467187, 0.49213, 0.077083, 0.130556],
  [2.0, 0.123438, 0.398611, 0.030208, 0.052778]],
 [[2.0, 0.7125, 0.564815, 0.095833, 0.157407],
  [2.0, 0.234896, 0.442593, 0.029167, 0.059259]],
 [[2.0, 0.532813, 0.496296, 0.052083, 0.094444]],
 [[3.0, 0.446875, 0.472685, 0.120833, 0.232407]],
 [[2.0, 0.232552, 0.522685, 0.085938, 0.152778],
  [3.0, 0.067708, 0.426852, 0.008333, 0.012963]],
 [[2.0, 0.779948, 0.562963, 0.090104, 0.155556],
  [2.0, 0.933333, 0.752778, 0.133333, 0.412963]],
 [[2.0, 0.552865, 0.502778, 0.085938, 0.146296],
  [2.0, 0.411458, 0.460648, 0.044792, 0.086111]],
 [[2.0, 0.404167, 0.459259, 0.047917, 0.083333],
  [2.0, 0.54349, 0.49537, 0.069271, 0.146296],
  [3.0, 0.798698, 0.511574, 0.042188, 0.093519],
  [2.0, 0.929688, 0.54213, 0.067708, 0.130556]] ,
 [[3.0, 0.760677, 0.39213, 0.070312, 0.147222]],
 [[3

In [66]:
# Compile the model
learning_rate = 0.001
optimizer = Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)
# tf.convert_to_tensor(input_boxes)
# tf.convert_to_tensor(output_boxes)
# Train the amodal regressor
epochs = 50
batch_size = 32
model.fit(input_boxes, output_boxes, epochs=epochs, batch_size=batch_size)

# Use the trained model to generate boxes for occluded riders (you'll need a separate dataset with occluded riders)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: ignored

In [ ]:
# Save the trained model for future use
model.save('amodal_regressor_model.h5')